# 导入环境

In [1]:
import mindnlp
import mindspore

# mindspore.set_context(pynative_synchronize=True)
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

[WARNING] DEVICE(1065413,ffff968db020,python):2025-08-16-13:12:49.008.926 [mindspore/ccsrc/utils/dlopen_macro.h:165] DlsymAscend] Dynamically load symbol aclprofGetSupportedFeaturesV2 failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libmsprofiler.so: undefined symbol: aclprofGetSupportedFeaturesV2
[WARNING] DEVICE(1065413,ffff968db020,python):2025-08-16-13:12:49.009.049 [mindspore/ccsrc/utils/dlopen_macro.h:165] DlsymAscend] Dynamically load symbol aclrtEventGetTimestamp failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclrtEventGetTimestamp
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal 

In [2]:
!wget https://openi.pcl.ac.cn/mindnlp/self-llm/raw/branch/master/dataset/huanhuan.json -O huanhuan.json --no-check-certificate

--2025-08-16 13:13:02--  https://openi.pcl.ac.cn/mindnlp/self-llm/raw/branch/master/dataset/huanhuan.json
Resolving openi.pcl.ac.cn (openi.pcl.ac.cn)... 218.17.115.247, 120.237.18.54, 210.22.22.132
Connecting to openi.pcl.ac.cn (openi.pcl.ac.cn)|218.17.115.247|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘huanhuan.json’

huanhuan.json           [          <=>       ] 914.84K   454KB/s    in 2.0s    

2025-08-16 13:13:05 (454 KB/s) - ‘huanhuan.json’ saved [936793]



In [3]:
# 将JSON文件转换为CSV文件
df = pd.read_json('./huanhuan.json')
ds = Dataset.from_pandas(df)

In [4]:
ds[:3]

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。'],
 'input': ['', '', ''],
 'output': ['嘘——都说许愿说破是不灵的。', '你们俩话太多了，我该和温太医要一剂药，好好治治你们。', '出来走走，也是散心。']}

# 处理数据集

In [5]:
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-7B-Instruct', use_fast=False, trust_remote_code=True)
tokenizer

Qwen2Tokenizer(name_or_path='Qwen/Qwen2.5-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, norm

In [6]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 3729/3729 [00:13<00:00, 279.28 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3729
})

In [8]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n<|im_start|>user\n小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>\n<|im_start|>assistant\n嘘——都说许愿说破是不灵的。<|endoftext|>'

In [9]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'你们俩话太多了，我该和温太医要一剂药，好好治治你们。<|endoftext|>'

# 创建模型

In [ ]:
import torch

model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-7B-Instruct', torch_dtype=torch.float16, attn_implementation='eager')
model = model.npu()

Loading checkpoint shards: 100%|██████████| 4/4 [00:32<00:00,  8.19s/it]


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


In [ ]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [ ]:
model.dtype

mindspore.float16

# lora 

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'gate_proj', 'v_proj', 'o_proj', 'up_proj', 'q_proj', 'down_proj', 'k_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [ ]:
model = get_peft_model(model, config)
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='Qwen/Qwen2.5-7B-Instruct', revision=None, inference_mode=False, r=8, target_modules={'gate_proj', 'v_proj', 'o_proj', 'up_proj', 'q_proj', 'down_proj', 'k_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [ ]:
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


# 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="./output/Qwen2.5_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, 
    learning_rate=1e-4,
    save_on_each_node=True,
    # fp16=True,
    # gradient_checkpointing=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.accelerator.state

Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: device(type=npu, index=0)

Mixed precision type: no

In [ ]:
trainer.train()

[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:07:37.656.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:07:40.106.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.


embed_tokens True
position_embeddings True


[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:07:40.276.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:07:40.293.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:07:40.310.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:07:40.326.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package vers

decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer True
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False


[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:07:40.481.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:07:40.498.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:07:40.515.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:07:40.532.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package vers

...

[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:12:45.846.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:12:45.856.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:12:45.875.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:12:45.889.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package vers

embed_tokens True
position_embeddings False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer True
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False


[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:12:46.600.00 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:12:46.740.00 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:12:46.880.00 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:12:46.101.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version

decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
decoder_layer False
norm True


[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:12:46.268.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.
[WARNING] ME(1054275:281473393594400,MainProcess):2025-08-16-13:12:46.273.000 [mindspore/run_check/_check_version.py:305] The version 7.7 used for compiling the custom operator does not match Ascend AI software package version 7.6 in the current environment.


# 合并加载模型

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = 'Qwen/Qwen2.5-7B-Instruct'
lora_path = './output/Qwen2.5_instruct_lora/checkpoint-702' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "你是谁？"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))